# PSM
## Notatki bardzo luźne, Dominik Stańczak
[Strona przedmiotu](http://if.pw.edu.pl/~agatka/psm.html)

Wykłady 8:30 bez przerwy

[Instrukcje do labków](http://fizyka.if.pw.edu.pl/~labe/index.php/Laboratorium_PSM)

## [Wykład 1](http://if.pw.edu.pl/~agatka/psm/wyklad_01.pdf)


Prezentacje ogólnie będą.
Przedmiot jest dość wredny bo trzeba się uczyć instrukcji, ale raczej do ogarnięcia.

99% elektroniki jest w mikrokontrolerach. Trochę jak 99% materii we wszechświecie to plazma. Więc mikrokontrolery to taka plazma, tylko inna.

Książki:
* Francuz, *Język  C dla mikrokontrolerów AVR* - bardziej zaawansowana pod względem C
* Mirosław Kardaś, *Mikrokontrolery AVR, język C* - mniej zaawansowana

Koszt płytki do ćwiczeń własnych - Atmel AVR - rzędu 100 zł

Ocena - $\frac{2}{3} $ lab $ + \frac{1}{3}$ wykład, obie muszą być pozytywne
Kolokwium w 1/4 semestru i 1/2 semestru

Konwersja z binarki do hexa, najprościej przez DEC: $0123456789ABCDEF, 0xF = 15, 0x10 = 16, 255 = 0b = 0xFF$
Pojedynczy bajt to jedno 8bitowe **słowo**, np. $0xE4 = 4+15*16 = 0b1110|0100$

> TARDIS, nie wiem czy kojarzycie...

[Magistrala](https://pl.wikipedia.org/wiki/Magistrala_komunikacyjna), z angielska [bus](https://en.wikipedia.org/wiki/Bus_(computing) - system komunikacji komputera\mikrokontrolera z zewnątrznym urządzeniem. np PCI Express jest magistralą

# [Wykład 2](http://if.pw.edu.pl/~agatka/psm/wyklad_02.pdf)

W mikrokontrolerach lepiej definiować zmienne na zewnątrz funkcji, przed mainem itd. Szybciej to śmiga.

Int branchuje się po zainclude'owaniu <stdint.h> na *(u)*int[8,16,32,64]_t - lepiej stosować takie typy w celu oszczędzenia na pamięci.

```c
#include <stdint.h>
uint8_t x = 1;
```

Struktura if-not-define:
```c
#ifndef LCD_H_
#define LCD_H_
{
    void led_init(void);
}
#endif
```
Żeby uniknąć definiowania funkcji (pamięć!), można zrobić tak
```c
#define LICZ(a,b) (a+b)
```
trzeba pamiętać o kolejności wykonywania poleceń. nawiasy są tu git pomysłem.

Zamiast tworzenia zmiennych typu const:
```c
#define LED 0b00010000
PORTC = LED;
```

Kilka ograniczeń Atmegi:
* *atmega słabo obługuje floaty*
* **doubli nie ma w ogóle.**
* Nie da się zapisać pojedynczego bitu (toteż nie ma zysku na definiowaniu 1-bitowego boola. W celu oszczędzenia miejsca można zamiast tego zebrać bity w struktury. Na prezentacji pod "Bity zapisujemy tak":

```c
struct liczba {
unsigned int flaga_1 :1;        // :1 - miejsce na 1 bit
unsigned int flaga_2 :1;
unsigned int flaga_dwubitowa :2;
} x; //x jest typu liczba

x.flaga_1 = 1;
x.flaga_2 = 0;
x.flaga_dwubitowa = 3; //3 = 0b11, akurat się zmieści
```

Przykładowy kod blinkowania:
```c
#define CZAS 10
int main(void)
{
    DDRA = 0xFF; //wszystkie piny A na out
    for(;;) //fikuśny while 1
    {
        PORTA ^= 0xFF; //odwracamy wszystkie bity
        _delay_ms(CZAS)
    }
    return;
}
```


### Operacje bitowe
* Not: ~
* And: &
* Or: |
* Nor: ^
* Bitowe w lewo: <<
* Bitowe w prawo: >>

```c
int x    = 10111100;
int mask = 11101111;
//x&mask = 10101100;
int m2   = 01000000;
//x|m2   = 11111100;

int z = x << 5;
// int z = 10000000;
```

In [14]:
x = 0b10111100
x, bin(x), hex(x)

(188, '0b10111100', '0xbc')

In [11]:
z = x<<5
z, bin(z), hex(z)

(6016, '0b1011110000000', '0x1780')

Wynik dynamic typingu w Pythonie!

In [25]:
y = 0b0000000000000000000000000001
y

1

Automatycznie dopasowuje liczbę potrzebnych bitów.

In [70]:
import numpy as np

x = np.array([0b1011100], dtype=np.uint8)
y = x << 5
z = x >> 5 #usuwa 11100
bin = lambda x: np.binary_repr(x, width=8) #bo pracuję na uint8
bin(x), type(bin(x)), bin(y), bin(z), z.dtype

('01011100', str, '10000000', '00000010', dtype('uint8'))

In [72]:
mask = np.array([1], dtype = np.uint8)
print(bin(x),
    bin(mask),
    bin(mask<<5),
    bin(x|(mask<<5)),
    bin(mask<<4),
    bin(x&(mask<<4)),
    bin(~mask),
    bin(~(mask<<4)),
    bin(x & ~(mask <<4)),
    sep='\n')

01011100
00000001
00100000
01111100
00010000
00010000
11111110
11101111
01001100


```c
// wszystkie bity na 1
DDRA = 0xFF;

//0, 2, 7 portu A na wyjściowe (na 1), reszta na wejściowe (0)
DDRA = (1<<7) | (1<<2) & 1;
// można też
DDRA = (1<<PA7) | (1<<PA2) & (1<<PA0);

//0, 7 poru A na 1 *bez modyfikacji reszty ( |= )*
PORTA |= (1<<0) | (1<<7);

//odczyt 4 najbardziej znaczących bitów z A
data = PINA >> 4;
//przekombinowane, z wykładu:
data = (PINA & (0xF0)) >> 4;

//odracamy pin 3:
PORTA ^= (1<<3);

//check czy bit 3 jest ustawiony
if(PINA & (1<<3));
//check czy bit 3 jest wyzerowany
if(! ( PINA & (1<<3) ) ); //WAŻNE IŻ !, nie ~
```
### Zadanko domowe:
```c
x = 0b00000011;
x = (x<<1 | x>>3);
```

In [127]:
x = np.array(0b1111, dtype=np.uint8);
bin = lambda x: np.binary_repr(x, width=8) #bo pracuję na uint8
print("x: {}".format(bin(x)))

for i in range(24):
    i = i%8
    if (i==0):
        print("Kolejna rotacja!")
    shift_forward = (x<<i).astype(np.uint8)
    shift_minus = (x>>(8-i)).astype(np.uint8)
    result = shift_1 | shift_minus
    print(bin(shift_forward), bin(shift_minus), bin(shift_forward | shift_minus))

x: 00001111
Kolejna rotacja!
00001111 00000000 00001111
00011110 00000000 00011110
00111100 00000000 00111100
01111000 00000000 01111000
11110000 00000000 11110000
11100000 00000001 11100001
11000000 00000011 11000011
10000000 00000111 10000111
Kolejna rotacja!
00001111 00000000 00001111
00011110 00000000 00011110
00111100 00000000 00111100
01111000 00000000 01111000
11110000 00000000 11110000
11100000 00000001 11100001
11000000 00000011 11000011
10000000 00000111 10000111
Kolejna rotacja!
00001111 00000000 00001111
00011110 00000000 00011110
00111100 00000000 00111100
01111000 00000000 01111000
11110000 00000000 11110000
11100000 00000001 11100001
11000000 00000011 11000011
10000000 00000111 10000111


# [Wykład 3](http://if.pw.edu.pl/~agatka/psm/wyklad_03.pdf) 8 marca

## ~~goto~~ Przerwania

No... takie ~~zło~~ goto z automatycznym powrotem do miejsca jego wywołania.

Mogą je powodować zmiany odczytów z pinów albo sam kod programu.

Warunki:
* Globalny bit przerwań **SREG**-bit 7 musi być ustawiony
* Bit tego konkretnego przerwania które odpalamy musi być ustawiony
* Zaszedł trigger przerwania



In [ ]:
m